In [1]:
import sys
sys.path.append("..")


In [3]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("../grasp_analysis_results.csv")

# Function to calculate success rates
def calculate_success_rates(data):
    total = len(data)
    graspable = (data['is_graspable'] == True).sum() / total * 100
    collision_free = (data['has_collision'] == False).sum() / total * 100
    successful = ((data['is_graspable'] == True) & (data['has_collision'] == False)).sum() / total * 100
    
    return {
        'Total samples': total,
        'Graspable rate': f"{graspable:.2f}%",
        'Collision-free rate': f"{collision_free:.2f}%",
        'Overall success rate': f"{successful:.2f}%"
    }

# Split data into training and non-training sets
training_data = df[df['is_used_in_training'] == True]
non_training_data = df[df['is_used_in_training'] == False]

# Calculate and print results
print("Training Data Results:")
training_results = calculate_success_rates(training_data)
for metric, value in training_results.items():
    print(f"{metric}: {value}")

print("\nNon-Training Data Results:")
non_training_results = calculate_success_rates(non_training_data)
for metric, value in non_training_results.items():
    print(f"{metric}: {value}")

Training Data Results:
Total samples: 0
Graspable rate: nan%
Collision-free rate: nan%
Overall success rate: nan%

Non-Training Data Results:
Total samples: 8740864
Graspable rate: 96.81%
Collision-free rate: 84.92%
Overall success rate: 81.77%


/var/folders/br/r63rm8nn1zn3hx94h8yqsrth0000gn/T/ipykernel_57540/1173247341.py:9: RuntimeWarning: invalid value encountered in scalar divide
  graspable = (data['is_graspable'] == True).sum() / total * 100
/var/folders/br/r63rm8nn1zn3hx94h8yqsrth0000gn/T/ipykernel_57540/1173247341.py:10: RuntimeWarning: invalid value encountered in scalar divide
  collision_free = (data['has_collision'] == False).sum() / total * 100
/var/folders/br/r63rm8nn1zn3hx94h8yqsrth0000gn/T/ipykernel_57540/1173247341.py:11: RuntimeWarning: invalid value encountered in scalar divide
  successful = ((data['is_graspable'] == True) & (data['has_collision'] == False)).sum() / total * 100


In [4]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("../grasp_analysis_results.csv")

# First let's look at the value counts of is_used_in_training
print("Distribution of is_used_in_training:")
print(df['is_used_in_training'].value_counts())
print("\nUnique values in is_used_in_training:")
print(df['is_used_in_training'].unique())

# Check the data types
print("\nData types of columns:")
print(df.dtypes)

# Look at a few rows
print("\nFirst few rows:")
print(df.head())

Distribution of is_used_in_training:
is_used_in_training
False    8740864
Name: count, dtype: int64

Unique values in is_used_in_training:
[False]

Data types of columns:
mesh_path               object
is_used_in_training       bool
dataset_mesh_scale     float64
normalization_scale    float64
has_collision             bool
min_distance           float64
is_graspable              bool
grasp_translation       object
grasp_rotation          object
centroid                object
dtype: object

First few rows:
                                           mesh_path  is_used_in_training  \
0  data/meshes/Poster/f6ef87c28c31be768ba2126777c...                False   
1  data/meshes/Poster/f6ef87c28c31be768ba2126777c...                False   
2  data/meshes/Poster/f6ef87c28c31be768ba2126777c...                False   
3  data/meshes/Poster/f6ef87c28c31be768ba2126777c...                False   
4  data/meshes/Poster/f6ef87c28c31be768ba2126777c...                False   

   dataset_mesh_scale  no

In [5]:
training_data = df[df['is_used_in_training'] == True]
print(training_data.head())

Empty DataFrame
Columns: [mesh_path, is_used_in_training, dataset_mesh_scale, normalization_scale, has_collision, min_distance, is_graspable, grasp_translation, grasp_rotation, centroid]
Index: []


In [ ]:
import json

with open("../logs/checkpoints/run_20250202_233846/used_grasp_files.json", "r") as f:
    used_files = json.load(f)

print(used_files[:10])

['PosterBed_189064a8df83ddc77a89a0a7fbbe0a6f_0.0014520024942939636.h5', 'EndTable_ece7f04700c5767a4cbf9a02e3342127_0.003536836430918311.h5', 'DiningTable_aeb34b796044d26f2705f56670a32884_0.0011257601249697977.h5', 'Laptop_d555f1c62be56e92fd99188d937624c5_0.01620609712837213.h5', 'Sink_5c07f7040e822a42fdb729ff38860b34_0.011044232918568831.h5', 'Dresser_78f812dd910be1dd5a0e5699b3040a35_0.0036026507719122496.h5', 'Helicopter_3b4cbd4fd5f6819bea4732296ea50647_0.00017694834146319866.h5', 'CellPhone_4c992058eca403d9980a39e0273687b1_0.036959886019561294.h5', 'ToiletPaper_260949513aaba2eb90cadbd65232985b_0.00016124514791527317.h5', 'PaperBox_ffacfd96ecee72ba9aefc6fb585e9617_0.0008284578658023282.h5']


In [10]:
import pandas as pd
from pathlib import Path
import numpy as np

def extract_info_from_cache_files(used_files):
    """Extract info from all cache files at once."""
    cache_info = []
    for filename in used_files:
        parts = filename[:-3].split("_")  # Remove .h5
        cache_info.append({
            'item_name': "_".join(parts[:-2]),
            'item_id': parts[-2],
            'scale': float(parts[-1])
        })
    return pd.DataFrame(cache_info)

def extract_info_from_mesh_paths(df):
    """Extract item_name and item_id from all mesh paths at once."""
    paths = df['mesh_path'].apply(Path)
    return pd.DataFrame({
        'item_name': paths.apply(lambda p: p.parent.name),
        'item_id': paths.apply(lambda p: p.stem),
        'scale': df['dataset_mesh_scale']
    })

# Read the CSV
df = pd.read_csv("../grasp_analysis_results.csv")

# Load used_files (replace this with your loading method)
# used_files = ["item_name_item_id_scale.h5", ...]

# Extract info from both sources
cache_df = extract_info_from_cache_files(used_files)
mesh_df = extract_info_from_mesh_paths(df)

# Create boolean mask for matching entries
matches = pd.merge(
    mesh_df.assign(idx=np.arange(len(mesh_df))), 
    cache_df,
    on=['item_name', 'item_id'],
    suffixes=('_mesh', '_cache')
)
matches = matches[np.isclose(matches['scale_mesh'], matches['scale_cache'], rtol=1e-9)]

# Update is_used_in_training using the matches
df['is_used_in_training'] = False
df.loc[matches['idx'], 'is_used_in_training'] = True

# Save the updated CSV
df.to_csv("../grasp_analysis_results_updated.csv", index=False)

In [11]:
import pandas as pd

def analyze_grasp_results(df, label=""):
    """Analyze grasp results for a given dataset."""
    total = len(df)
    if total == 0:
        print(f"\n{label} Results: No samples found")
        return
    
    metrics = {
        'Graspable rate': (df['is_graspable'] == True).mean() * 100,
        'Collision-free rate': (df['has_collision'] == False).mean() * 100,
        'Overall success rate': ((df['is_graspable'] == True) & 
                               (df['has_collision'] == False)).mean() * 100
    }
    
    print(f"\n{label} Results (Total samples: {total:,}):")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.2f}%")
    
    print("\nDetailed breakdown:")
    print(pd.crosstab(df['is_graspable'], df['has_collision']))

# Read the CSV
df = pd.read_csv("../grasp_analysis_results_updated.csv")

# Print overall statistics
total = len(df)
training = df['is_used_in_training'].sum()
print(f"Dataset Overview:")
print(f"Total samples: {total:,}")
print(f"Training samples: {training:,} ({training/total*100:.2f}%)")
print(f"Non-training samples: {total-training:,} ({(total-training)/total*100:.2f}%)")

# Analyze both sets
analyze_grasp_results(df[df['is_used_in_training']], "Training Data")
analyze_grasp_results(df[~df['is_used_in_training']], "Non-Training Data")

Dataset Overview:
Total samples: 8,740,864
Training samples: 7,921,664 (90.63%)
Non-training samples: 819,200 (9.37%)

Training Data Results (Total samples: 7,921,664):
Graspable rate: 97.08%
Collision-free rate: 85.74%
Overall success rate: 82.85%

Detailed breakdown:
has_collision    False    True 
is_graspable                   
False           228807     2503
True           6562854  1127500

Non-Training Data Results (Total samples: 819,200):
Graspable rate: 94.24%
Collision-free rate: 76.98%
Overall success rate: 71.31%

Detailed breakdown:
has_collision   False   True 
is_graspable                 
False           46495     681
True           584151  187873


In [13]:
import pandas as pd

def analyze_grasp_results(df, label=""):
    """Analyze grasp results for a given dataset."""
    total = len(df)
    if total == 0:
        print(f"\n{label} Results: No samples found")
        return
    
    metrics = {
        'Graspable rate': (df['is_graspable'] == True).mean() * 100,
        'Collision-free rate': (df['has_collision'] == False).mean() * 100,
        'Overall success rate': ((df['is_graspable'] == True) & 
                               (df['has_collision'] == False)).mean() * 100
    }
    
    print(f"\n{label} Results (Total samples: {total:,}):")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.2f}%")
    
    print("\nDetailed breakdown:")
    print(pd.crosstab(df['is_graspable'], df['has_collision']))

# Read the CSV
df = pd.read_csv("../training.csv")

# Print overall statistics
total = len(df)
training = df['is_used_in_training'].sum()
print(f"Dataset Overview:")
print(f"Total samples: {total:,}")
print(f"Training samples: {training:,} ({training/total*100:.2f}%)")
print(f"Non-training samples: {total-training:,} ({(total-training)/total*100:.2f}%)")

# Analyze both sets
analyze_grasp_results(df[df['is_used_in_training']], "Training Data")
analyze_grasp_results(df[~df['is_used_in_training']], "Non-Training Data")

Dataset Overview:
Total samples: 1,843
Training samples: 0 (0.00%)
Non-training samples: 1,843 (100.00%)

Training Data Results: No samples found

Non-Training Data Results (Total samples: 1,843):
Graspable rate: 100.00%
Collision-free rate: 98.53%
Overall success rate: 98.53%

Detailed breakdown:
has_collision  False  True 
is_graspable               
True            1816     27


In [17]:
import pandas as pd
import torch
from core.visualize import check_collision

# Load the CSV
df = pd.read_csv('../training.csv')

# Filter for grasps that have collision but are graspable
interesting_grasps = df[
    (df['has_collision'] == True) & 
    (df['is_graspable'] == True) 
    # & 
    # (~df['was_skipped'])
]

print(f"Found {len(interesting_grasps)} grasps with collision that are graspable")

# Function to convert string representation of list back to tensor
def str_to_tensor(s):
    # Remove brackets and split by comma
    values = eval(s)  # Be careful with eval - use only on trusted data
    return torch.tensor(values)

# Visualize each interesting grasp
for idx, grasp in interesting_grasps.iterrows():
    print(f"\nVisualizing grasp {idx} for mesh: {grasp['mesh_path']}")
    
    # Convert stored data back to tensors
    rotation = str_to_tensor(grasp['grasp_rotation']).unsqueeze(0)  # Add batch dimension
    translation = str_to_tensor(grasp['grasp_translation']).unsqueeze(0)
    
    # Call check_collision to get the scene
    has_collision, scene, min_distance, is_graspable = check_collision(
        rotation,
        translation,
        "../"+grasp['mesh_path'],
        grasp['dataset_mesh_scale']
    )
    
    # Show the scene
    scene.show()
    
    break

Found 27 grasps with collision that are graspable

Visualizing grasp 20 for mesh: data/meshes/Pizza/caca4c8d409cddc66b04c0f74e5b376e.obj
